In [1]:
import cv2
import numpy as np
import os
import tensorflow as tf

2025-10-12 15:44:59.459735: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-10-12 15:44:59.566605: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-10-12 15:45:00.435026: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
image_path = os.getcwd() + "/data/FER-2013"
img_size = (96,96)
batch_size = 8

In [3]:
train = tf.keras.utils.image_dataset_from_directory(image_path + "/train",
                                                    seed = 0,
                                                    image_size = img_size,
                                                    subset = 'training',
                                                    validation_split = 0.2,
                                                    color_mode = "grayscale",
                                                    batch_size = batch_size)

val = tf.keras.utils.image_dataset_from_directory(image_path + "/train",
                                                    seed = 0,
                                                    image_size = img_size,
                                                    subset = 'validation',
                                                    validation_split = 0.2,
                                                    color_mode = "grayscale",
                                                    batch_size = batch_size)

test = tf.keras.utils.image_dataset_from_directory(image_path + "/test",
                                                    image_size = img_size,
                                                    color_mode = "grayscale",
                                                    batch_size = batch_size)

Found 28709 files belonging to 7 classes.
Using 22968 files for training.


I0000 00:00:1760264101.604362   54168 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 6155 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 4060 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.9


Found 28709 files belonging to 7 classes.
Using 5741 files for validation.
Found 7178 files belonging to 7 classes.


In [4]:
# Augmentation pipeline
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
    tf.keras.layers.RandomContrast(0.1),
])

# Apply only to training set
AUTOTUNE = tf.data.AUTOTUNE

augmented_train = (train
         .shuffle(1000)
         .map(lambda x, y: (data_augmentation(x, training=True), y),
              num_parallel_calls=AUTOTUNE)
         .prefetch(AUTOTUNE))

val = val.prefetch(AUTOTUNE)
test = test.prefetch(AUTOTUNE)

In [5]:
train_expanded = train.concatenate(augmented_train)

In [6]:
len(train_expanded)*8

45936

In [7]:
# model = tf.keras.models.Sequential([
#     tf.keras.layers.Input(shape=(48,48,1)),
#     tf.keras.layers.Conv2D(filters=32,
#                            kernel_size = (3,3),
#                            activation = 'relu'),
#     tf.keras.layers.Conv2D(filters=64,
#                            kernel_size = (3,3),
#                            activation = 'relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Conv2D(filters=128,
#                            kernel_size = (3,3),
#                            activation = 'relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Conv2D(filters=128,
#                            kernel_size = (3,3),
#                            activation = 'relu'),
#     tf.keras.layers.MaxPooling2D(2,2),
#     tf.keras.layers.Dropout(0.25),

#     tf.keras.layers.Flatten(),
#     tf.keras.layers.Dense(1024, activation='relu'),
#     tf.keras.layers.Dropout(0.5),

#     tf.keras.layers.Dense(7, activation='softmax')
# ])

model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(96, 96, 1)),

    tf.keras.layers.Rescaling(1./255),
    tf.keras.layers.Conv2D(
                    filters=96,
                    kernel_size=(3,3),
                    activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(
                    filters=256,
                    kernel_size=(3,3),
                    activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Conv2D(
                    filters=384,
                    kernel_size=(3,3),
                    activation='relu'),
    tf.keras.layers.Conv2D(
                    filters=384,
                    kernel_size=(3,3),
                    activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(
                    filters=256,
                    kernel_size=(3,3),
                    activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(4096, activation='relu'),
    tf.keras.layers.Dense(7, activation='softmax')
])

In [8]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy('mixed_float16')

In [ ]:
history = model.fit(
    train_expanded,
    validation_data = val,
    epochs = 50,
    callbacks = [tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True)]
)

Epoch 1/50


2025-10-12 15:45:04.472877: I external/local_xla/xla/service/service.cc:163] XLA service 0x7f667000c220 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2025-10-12 15:45:04.472899: I external/local_xla/xla/service/service.cc:171]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-10-12 15:45:04.510197: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-10-12 15:45:04.776876: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91301
2025-10-12 15:45:05.861933: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_2544', 8 bytes spill stores, 8 bytes spill loads

2025-10-12 15:45:06.222386: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:34

  10/2871 ━━━━━━━━━━━━━━━━━━━━ 49s 17ms/step - accuracy: 0.1307 - loss: 23.9793

I0000 00:00:1760264110.946492   54372 device_compiler.h:196] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


 196/2871 ━━━━━━━━━━━━━━━━━━━━ 44s 16ms/step - accuracy: 0.2062 - loss: 6.6678

KeyboardInterrupt: 

In [ ]:
model.save("model_fer.keras")